# scVI for normalized_gene_expression

scVI is run here to obtain a more smoothed gene expression landscape. This is mostly done because for cells with 0 expression in some genes correlations of pulls to these driver genes does not work well. 

In [1]:
import numpy as np
import anndata
import scanpy as sc
import scvi

/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
!pip list

Package                      Version
---------------------------- ---------------------
absl-py                      1.4.0
aiohttp                      3.8.4
aiosignal                    1.3.1
anndata                      0.8.0
anyio                        3.6.2
apturl                       0.5.2
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
arrow                        1.2.3
asn1crypto                   0.24.0
asttokens                    2.2.1
astunparse                   1.6.3
async-lru                    2.0.2
async-timeout                4.0.2
attrs                        17.4.0
Automat                      0.6.0
Babel                        2.12.1
backcall                     0.2.0
beautifulsoup4               4.6.0
bleach                       2.1.2
blinker                      1.4
Brlapi                       0.6.6
cached-property              1.5.2
cachetools                   5.3.0
cellrank                     1.5.2.dev174+gad1248d
certifi           


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [3]:
Path="/home/mgander/mouse_atlas/data/"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

# Running scVI over all time points of interest

In [9]:
number_of_hvgs=3000

for i in [7,10,11]:
    ts0=ts[i]
    ts1=ts[i+1]
    print(ts0)
    # Prepare the data set
    adata=anndata.read(f'{Path}/Comb_anndatas/adata_{ts0}_{ts1}.h5ad')
    adata=adata[adata.obs['day']==min(set(adata.obs['day']))].copy()
    avar=adata.var.copy()
    adata=adata.raw.to_adata()
    adata.var=avar.copy()
    adata.layers["counts"] = adata.X.copy()
    sc.pp.highly_variable_genes(adata, n_top_genes=number_of_hvgs, subset=True, layer="counts", flavor="seurat_v3")


    # Run scVI
    scvi.model.SCVI.setup_anndata(adata, layer="counts")
    model = scvi.model.SCVI(adata)
    model.train()
    adata.obsm["X_scVI"]=model.get_latent_representation()
    adata.layers["scvi_normalized"] = model.get_normalized_expression(library_size=10e4)
    adata.write(f'{Path}/Comb_anndatas/scVI_anndatas/scvi_adata_{ts0}.h5ad', compression='gzip')

E7.0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 400/400: 100%|████████████████████████████████████████████████████| 400/400 [15:05<00:00,  2.23s/it, v_num=1, train_loss_step=1.53e+3, train_loss_epoch=1.55e+3]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|████████████████████████████████████████████████████| 400/400 [15:05<00:00,  2.26s/it, v_num=1, train_loss_step=1.53e+3, train_loss_epoch=1.55e+3]
E7.75


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 400/400: 100%|████████████████████████████████████████████████████| 400/400 [14:40<00:00,  2.21s/it, v_num=1, train_loss_step=1.61e+3, train_loss_epoch=1.56e+3]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|████████████████████████████████████████████████████| 400/400 [14:40<00:00,  2.20s/it, v_num=1, train_loss_step=1.61e+3, train_loss_epoch=1.56e+3]


In [4]:
# For E13.5 we have much more cells and noisier data, hence I use 6k hvgs

number_of_hvgs=6000

for i in [16]:
    ts0=ts[i]
    ts1=ts[i+1]
    print(ts0)
    # Prepare the data set
    adata=anndata.read(f'{Path}/Comb_anndatas/adata_{ts0}_{ts1}.h5ad')
    adata=adata[adata.obs['day']==min(set(adata.obs['day']))].copy()
    avar=adata.var.copy()
    adata=adata.raw.to_adata()
    adata.var=avar.copy()
    adata.layers["counts"] = adata.X.copy()
    sc.pp.highly_variable_genes(adata, n_top_genes=number_of_hvgs, subset=True, layer="counts", flavor="seurat_v3")


    # Run scVI
    scvi.model.SCVI.setup_anndata(adata, layer="counts")
    model = scvi.model.SCVI(adata)
    model.train()
    adata.obsm["X_scVI"]=model.get_latent_representation()
    adata.layers["scvi_normalized"] = model.get_normalized_expression(library_size=10e4)
    adata.write(f'{Path}/Comb_anndatas/scVI_anndatas/scvi_adata_{ts0}.h5ad', compression='gzip')

E10.5


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████| 30/30 [21:20<00:00, 41.05s/it, v_num=1, train_loss_step=649, train_loss_epoch=679]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████| 30/30 [21:21<00:00, 42.70s/it, v_num=1, train_loss_step=649, train_loss_epoch=679]
